In [1]:
import AdventOfCode
import Debug.Trace

In [2]:
input <- dayLines 17

In [3]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.Attoparsec.ByteString.Char8 as C

parseXY = do
    C.string "x="
    x <- C.decimal
    C.string ", y="
    yl <- C.decimal
    C.string ".."
    yr <- C.decimal
    pure [(x,y) | y <- [yl..yr]]

parseYX = do
    C.string "y="
    y <- C.decimal
    C.string ", x="
    xl <- C.decimal
    C.string ".."
    xr <- C.decimal
    pure [(x,y) | x <- [xl..xr]]

import Control.Applicative

parseLine = parseXY <|> parseYX

points = concatMap (parsed parseLine) input

In [4]:
import Data.Function (on)
import Data.List

maxX = fst $ maximumBy (compare `on` fst) points
maxY = snd $ maximumBy (compare `on` snd) points


maxX
maxY
length points

594

1801

23894

In [5]:
minX = fst $ minimumBy (compare `on` fst) points
minY = snd $ minimumBy (compare `on` snd) points
minX
minY

314

3

In [6]:
import qualified Data.Map.Strict as Map

data Tile = Water | Clay deriving (Eq, Show)

pointMap = Map.fromList $ zip points (repeat Clay)

In [7]:
fall :: Map.Map (Int, Int) Tile -> (Int, Int) -> Int -> (Map.Map (Int, Int) Tile, (Int, Int))
fall pMap (x,y) maxy | y > maxy = (pMap, (x,y))
fall pMap (x,y) maxy = case Map.lookup (x,y) pMap of
    Nothing -> let
        pMap' = Map.insert (x,y) Water pMap
        in fall pMap' (x,y+1) maxy
    Just _ -> (pMap, (x,y))
    Just Water -> let
        (spreadL, coordL) = spread L pMap    (x-1,y-1) (x,y-2) maxy
        (spreadR, coordR) = spread R spreadL (x+1,y-1) (x,y-2) maxy
        in (spreadR, coordR)
    Just Clay -> let
        (spreadL, coordL) = spread L pMap    (x-1,y-1) (x,y-2) maxy
        (spreadR, coordR) = spread R spreadL (x+1,y-1) (x,y-2) maxy
        in (spreadR, coordR)

data Direction = L | R deriving (Eq, Show)

spread direction pMap (x,y) parent maxy | x < 0 = error "negative x value"
spread L pMap (x,y) parent@(pX, pY) maxy = case Map.lookup (x, y) pMap of -- the tile to be occupied
    Nothing -> case Map.lookup (x, y+1) pMap of -- the tile beneath the tile to be occupied
        Just _ -> let
            pMap' = Map.insert (x, y) Water pMap
            in spread L pMap' (x-1, y) parent maxy
        Nothing -> let
            pMap' = Map.insert (x, y) Water pMap
            in fall pMap' (x-1, y) maxy
    Just Clay -> fall pMap parent maxy
    Just Water -> (pMap, parent)
spread R pMap (x,y) parent@(pX, pY) maxy = case Map.lookup (x, y) pMap of -- the tile to be occupied
    Nothing -> case Map.lookup (x, y+1) pMap of -- the tile beneath the tile to be occupied
        Just _ -> let
            pMap' = Map.insert (x, y) Water pMap
            in spread R pMap' (x+1, y) parent maxy
        Nothing -> let
            pMap' = Map.insert (x, y) Water pMap
            in fall pMap' (x+1, y) maxy
    Just Clay -> fall pMap parent maxy
    Just Water -> (pMap, parent)

In [8]:
import qualified Data.Set as S
import qualified Data.Vector as V

pointSet = S.fromList points

row = V.replicate (maxX+1) ' '
tiles = V.replicate (maxY+1) row

display t = case t of
    Water -> 'w'
    Clay -> '#'

rowSet y rowLen pMap = [(x, display t) | x <- [0..(rowLen-1)], Map.member (x,y) pMap, let t = pMap Map.! (x,y)]

In [9]:
process grid pMap n | n >= V.length grid = grid
process grid pMap n = let
   pts = rowSet n (V.length (V.head grid)) pMap
   row' = (grid V.! n) V.// pts
   grid' = grid V.// [(n, row')]
   in process grid' pMap (n+1)

In [10]:
(step1, coord1) = fall pointMap (500,minY) maxY
coord1
processed = process tiles step1 0

(500,20)

In [11]:
undergroundMap = unlines $ V.toList $ fmap (intersperse ',' . V.toList) processed 

In [12]:
writeFile "map.txt" undergroundMap

In [13]:
handFilled <- readFile "map.csv"
still = filter (== '~') handFilled
flow = filter (== '|') handFilled

In [14]:
length $ still <> flow
length   still

36790

30765